In [0]:
pip install "snowflake-snowpark-python[pandas]" pandas


INFO: pip is looking at multiple versions of cryptography to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 111.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 156.8 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Not uninstalling cryptography at /databricks/python3/lib/python3.12/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-77abb302-d2ec-452b-ba8d-31c2222f7936
    Can't uninstall 'cryptography'. No files were found to uninstall.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
#restart kernel 
%restart_python

In [0]:
# snowpark_ingest.py
from snowflake.snowpark import Session
from snowflake.snowpark.types import StructType, StructField, StringType, DateType, IntegerType, DoubleType
import os
from datetime import datetime

# ------------- CONFIG -------------
connection_parameters = {
  "account": "ioijmaj-rr68892.snowflakecomputing.com>",         # e.g. xy12345.east-us-2.azure
  "user": "nikitha",
  "password": "Nikitha@21122003",                # or use key-pair auth
  "role": "ACCOUNTADMIN",
#   "warehouse": "<WAREHOUSE>",
  "database": "RETAIL_DB",
  "schema": "SALES"
}
STAGE_PATH = "@sales_stage_azure"  # or sales_stage_sas
CSV_FILE_PREFIX = "incoming/2025/10/"  # if you use subfolders
# -----------------------------------


In [0]:

session = Session.builder.configs(connection_parameters).create()

# Read CSVs from stage (wildcard)
csv_path = f"{STAGE_PATH}/{CSV_FILE_PREFIX}*.csv"

# If header present, Snowpark supports session.read.option
df = session.read.option("SKIP_HEADER", 1).option("FIELD_DELIMITER", ",").csv(csv_path)

# Inspect schema & basic transformations
# Example: rename columns to match table; adjust based on your CSV columns
# Suppose CSV columns: SalesID,SaleDate,ProductID,ProductName,StoreID,Region,Quantity,Price,TotalAmount

# Standardize column names
df = df.to_df("sales_id","sale_date","product_id","product_name","store_id","region","quantity","price","total_amount")

# Type cast
from snowflake.snowpark.functions import to_date, col
df = df.with_column("sale_date", to_date(col("sale_date"), "YYYY-MM-DD")) \
       .with_column("quantity", col("quantity").cast("INTEGER")) \
       .with_column("price", col("price").cast("DOUBLE")) \
       .with_column("total_amount", col("total_amount").cast("DOUBLE"))

# Remove duplicates (optional)
df = df.drop_duplicates(["sales_id"])

# Write into Snowflake table (append)
df.write.mode("append").save_as_table(["RETAIL_DB", "SALES", "MONTHLY_SALES"])

print("Ingest complete.")
session.close()


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:139)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:139)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:136)
	at scala.collection.immutable.Range.foreach(Range.scala:192)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:720)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:440)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:440)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can